In [ ]:
from io import BytesIO

import pandas as pd
import numpy as np
import requests

from bokeh.plotting import figure, show, output_notebook
from bokeh.palettes import viridis, Spectral10, Paired10, Set1_9, Set3_12, Dark2_8, Accent8
from bokeh.models import LabelSet, ColumnDataSource

output_notebook()

In [ ]:
def download_csv(csv_url):
    response = requests.get(csv_url)
    assert response.status_code == 200, 'Unable to download csv ' + csv_url
    return pd.read_csv(BytesIO(response.content)).dropna()

# swords csv:
#     sword: the name of the sword
#     weight: total weight of the sword
#     length: total length of the sword
#     balance: distance from the pommel to the balance point
#     pivot: distance from the pommel to the example pivot point
#     impact: distance from the pommel to the example impact point

# poi csv:
#     sword: the name of the sword
#     cm: distance from the pommel to the point of interest
#     label: the label of the point of interest

swords = download_csv('http://goo.gl/tqDCX0').set_index('sword')
poi = download_csv('http://goo.gl/wbJNoa')

In [ ]:
swords

In [ ]:
poi

In [ ]:
def graph_curves(only=None, show_labels=False):
    if only:
        swords_to_graph = only
    else:
        swords_to_graph = swords.index.values
    
    palette = Set1_9 + Dark2_8 + Accent8
    
    f = figure(title='Effective mass', width=900, y_range=(0, 2000))
    
    for i, sword_name in enumerate(swords_to_graph):
        sword = swords.ix[sword_name]
        
        l1 = abs(sword.impact - sword.balance)
        l2 = abs(sword.pivot - sword.balance)
        rectangle_area = l1 * l2

        def pivot_for_impact(impact):
            l1 = abs(impact - sword.balance)
            l2 = rectangle_area / l1
            pivot = sword.balance - l2
            return pivot

        def inertia_at_point(cm):
            impact = cm
            pivot = pivot_for_impact(impact)
            balance_to_impact = abs(impact - sword.balance)
            balance_to_pivot = abs(pivot - sword.balance)

            # to half the perimeter of our rectangle, which is just the length of the moment arm of the lever. 
            return sword.weight * (balance_to_pivot / (balance_to_pivot + balance_to_impact))

        points = pd.DataFrame()
        points['cm'] = [x for x in range(int(sword.length) + 1) if x != sword.balance]
        # TODO fix range int
        points['g'] = points['cm'].apply(inertia_at_point)
        
        f.line(
            points.cm.values, 
            points.g.values, 
            legend=None,
            line_width=1,
            line_color=palette[i],
        )
        
        sword_poi = poi[poi.sword == sword_name].copy()
        sword_poi['g'] = sword_poi['cm'].apply(inertia_at_point)
        
        f.circle(sword_poi.cm.values, 
                 sword_poi.g.values, 
                 legend=sword_name,
                 color=palette[i],
                 size=7)
        
        if show_labels:
            source = ColumnDataSource(data=sword_poi)
            labels = LabelSet(source=source, 
                              x='cm', y='g', text='label',
                              x_offset=5, y_offset=5)
            f.add_layout(labels)
        

    f.xaxis.axis_label = 'cm'
    f.yaxis.axis_label = 'g'
    
    show(f)

In [ ]:
graph_curves(show_labels=True)